# MLFlow

## 為什麼在 LlamaIndex 專案中使用 MLflow？

在使用 LlamaIndex 建立 RAG / 文件問答系統時，實務上會遇到幾個痛點：

1. **實驗不可重現**
   - embedding model 換了
   - chunk size / parser 換了
   - 向量庫內容更新了  
   👉 結果卻沒有清楚記錄「當時用的是哪一組設定」

2. **模型不是單一檔案**
   - LLM（Ollama / OpenAI / HF）
   - Embedding model
   - Vector Store（FAISS / Chroma / Weaviate）
   - Prompt / Query Engine  
   👉 傳統 `model.pkl` 的概念已經不夠用

3. **缺乏模型與資料的關聯追蹤**
   - 這個 index 是用哪一批資料建的？
   - 哪一個模型版本對應哪一個 dataset？

MLflow 在這裡的價值是：

- 📌 **實驗追蹤（Experiments & Runs）**
- 📦 **模型封裝與版本管理（Model Registry）**
- 🔗 **模型與資料的血緣關係（Data Lineage）**
- 🚀 **讓 LlamaIndex 模型可以被部署與重現**

本筆記示範如何：
- 使用 MLflow 追蹤 LlamaIndex Index / Query Engine
- 解決外部 Vector Store（FAISS）無法直接序列化的問題
- 將模型、程式碼、資料一起納入版本管理 

mlflow server --host 127.0.0.1 --port 8080

## Quick Start

In [ ]:
import os

os.chdir("../")

from src.ollama_connection import llama_index_ollama

model = "gpt-oss:120b-cloud"

ollama_llm = llama_index_ollama(model=model, temperature=0)

In [ ]:
import mlflow

# 通知mlflow要把紀錄送去哪裡
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("LlamaIndex")

### 快速的使用第一周的範例建立vectorstoreindex

讀取檔案

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext, Document 
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()

docs = loader.load("week_1/08物理.pdf")

doc_text = "\n\n".join([d.get_content() for d in docs])
docs = [Document(text=doc_text)]

建立VectorStoreIndex

In [ ]:
import faiss
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    transformations=[SimpleFileNodeParser()],
    embed_model=embed_model
)

# add documents to index
for doc in docs:
    index.insert(doc)

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    llm=ollama_llm,
    embed_model=embed_model,
)

Tracing

In [ ]:
mlflow.llama_index.autolog()

In [ ]:
response = await query_engine.aquery("基本力學的學習目標為何?")

log model

In [ ]:
with mlflow.start_run():
    model_info = mlflow.llama_index.log_model(
        index,
        name="index",
        engine_type="query",
        input_example="基本粒子的學習目標",
        registered_model_name="llama_index_basic" # 登記模型名稱
    )

### 為什麼直接 log_model(index) 會在 load_model 時失敗？

這不是 MLflow 或 LlamaIndex 的 bug，而是**設計上的限制**。

原因在於：

- FAISS index 是 **native binary object**
- 它無法被 pickle / cloudpickle 完整序列化
- MLflow 的 model logging 預設假設：
  >「模型本身可以被序列化或重建」

因此：
- `mlflow.llama_index.log_model(index)` 可以成功（因為只是在記錄）
- 但 `load_model()` 無法還原 FAISS index 的實體內容

👉 **正確的心智模型是**：

> MLflow 不負責「保存你的向量資料結構」  
> MLflow 負責「保存如何重建它的方法」

這也是為什麼官方與實務上都建議：
- 使用 **Python entry file（index.py）**
- 在 `load_context()` 中自行載入 FAISS / vector store

### 使用 Python file 作為 MLflow Model 的好處

當我們改用：

```python
mlflow.llama_index.log_model(
    "week_5/index.py",
    ...
)
```
實際上得到的是一個 PyFunc Model，它的優點包括：

✅ 可以自由控制：

Vector store 如何載入（FAISS / Chroma）

Embedding model 如何初始化

LLM 是否由外部注入（Ollama / OpenAI）

✅ 支援 multi-file dependency

code_paths 可以加入 loader、utils、config

非常適合真實專案結構

✅ 符合部署需求

MLflow Serving / Docker / K8s

不需要 notebook 環境

👉 這也是 MLflow 官方最推薦的做法

In [ ]:
with mlflow.start_run():
    model_info = mlflow.llama_index.log_model(
        "week_5/index.py",
        name="index",
        engine_type="query",
        input_example="基本粒子的學習目標",
        registered_model_name="llama_index_basic" # 登記模型名稱
    )

In [ ]:
index_loaded = mlflow.llama_index.load_model(model_uri="models:/llama_index_basic/1")

In [ ]:
index_loaded.as_query_engine(llm=ollama_llm).query("電磁學的學習目標")

Multi-file dependencies

In [ ]:
with mlflow.start_run():
    model_info = mlflow.llama_index.log_model(
        "week_5/index_multi.py",
        name="index",
        engine_type="query",
        input_example="熱力學的學習目標",
        code_paths=["week_5/load_index.py", "week_5/load_model.py"],  # Include dependency
        registered_model_name="llama_index_basic" # 登記模型名稱
    )

In [ ]:
index_loaded = mlflow.llama_index.load_model(model_uri="models:/llama_index_basic/2")

In [ ]:
index_loaded.as_query_engine(llm=ollama_llm).query("電磁學的學習目標")

模擬遠端執行

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# 列出特定模型的所有版本
versions = client.search_model_versions("name='llama_index_basic'")

for v in versions:
    print(f"Version: {v.version}, Stage: {v.current_stage}, Run ID: {v.run_id}")

給予模型Alias

In [ ]:
client.set_registered_model_alias(
    name="llama_index_basic",
    alias="champion",
    version="1"
)

### 模型描述

In [ ]:
client.update_registered_model(
    name="llama_index_basic",
    description="Basic LlamaIndex model for document Q&A"
)

### 模型 Tags（標籤）

MLflow 支援兩層 Tag：

- Registered model-level tags：整體模型的 metadata，例如用途、團隊等資訊。

- Model version-level tags：針對每個版本做不同註記，例如驗證狀態、效能資訊等。

In [ ]:
client.set_registered_model_tag(
    name="llama_index_basic",
    key="framework",
    value="llama-index"
)

client.set_registered_model_tag(
    name="llama_index_basic",
    key="owner",
    value="nlp-team"
)

給予特定模型版本 description 和 tag

In [ ]:
client.update_model_version(
    name="llama_index_basic",
    version="1",
    description="Uses OpenAI embeddings + FAISS vector store"
)

client.set_model_version_tag(
    name="llama_index_basic",
    version="1",
    key="embedding_model",
    value="text-embedding-3-large"
)

client.set_model_version_tag(
    name="llama_index_basic",
    version="1",
    key="index_type",
    value="faiss"
)

Get model version via alias

You cannot pass an alias directly into update_model_version() or set_model_version_tag().

Those APIs require an explicit version number.

So the workflow is:

Alias → resolve to version → update that version

In [ ]:
mv = client.get_model_version_by_alias(
    name="llama_index_basic",
    alias="champion")

In [ ]:
mv.version

In [ ]:
model_uri = "models:/llama_index_basic/1"

index_remote = mlflow.pyfunc.load_model(model_uri)

In [ ]:
model_info.registered_model_name

In [ ]:
mlflow.set_logged_model_tags(
        model_info.model_id, {"Training Info": "Basic LR model for iris data"}
    )
    

# MLflow數據追蹤

In [ ]:
import pandas as pd
import mlflow.data

# Load your data 第四周的 benchmark_evaluation
filename = "week_4/benchmark_evaluation.csv"
raw_data = pd.read_csv(filename)

In [ ]:
# Create a Dataset object
dataset = mlflow.data.from_pandas(
    raw_data, source=filename, name="benchmark"
)

## 紀錄數據的Metadata

In [ ]:
# Log the dataset to an MLflow run
with mlflow.start_run():
    mlflow.log_input(dataset, context="training")


### MLflow logs:

✔ Dataset name
✔ Dataset source
✔ Dataset schema (columns, types)
✔ Dataset digest / fingerprint
✔ Dataset context (training / validation / etc.)

🚫 NOT the actual data values

## 🔧 補充：MLflow Dataset ≠ Data Lake

⚠️ 常見誤解澄清：

- `mlflow.log_input()` **不會自動保存原始資料內容**
- 它的目的在於：
  - 建立「模型 ↔ 資料」的血緣關係
  - 確保實驗可追蹤、可稽核

👉 若你需要真正保存資料：
- 必須額外使用 `mlflow.log_artifact()`
- 或搭配外部資料儲存（S3 / GCS / DB）

這也是為什麼實務上常見做法是：
- Dataset lineage（metadata）→ MLflow
- Dataset 本體 → Artifact 或 Data Platform

Dataset Information and Metadata

In [ ]:
# Access dataset metadata
print(f"Dataset name: {dataset.name}")  # Defaults to "dataset" if not specified
print(
    f"Dataset digest: {dataset.digest}"
)  # Unique hash identifier (computed automatically)
print(f"Dataset source: {dataset.source}")  # DatasetSource object
print(
    f"Dataset profile: {dataset.profile}"
)  # Optional: implementation-specific statistics
print(f"Dataset schema: {dataset.schema}")  # Optional: implementation-specific schema

## 數據上傳

In [ ]:
with mlflow.start_run():
    mlflow.log_input(dataset, context="training")
    raw_data.to_parquet("week_5/benchmark.parquet")
    mlflow.log_artifact("week_5/benchmark.parquet", artifact_path="data")

In [ ]:
mlflow.artifacts.download_artifacts(run_id="628ba5d792924bc4987feec4c3bcbd16",
                                   artifact_path="data/benchmark.parquet",
                                   dst_path="week_5/downloads")

## 數據下載

### 模型綁定

In [ ]:
with mlflow.start_run() as run:
    # 1. Log dataset lineage
    dataset = mlflow.data.from_pandas(
        raw_data,
        source="week_4/benchmark_evaluation.csv",
        name="benchmark"
    )
    mlflow.log_input(dataset, context="training")

    # 2. Log dataset artifact
    raw_data.to_parquet("week_5/benchmark.parquet")
    mlflow.log_artifact("week_5/benchmark.parquet", artifact_path="data")

    model_info = mlflow.llama_index.log_model(
        "week_5/index_multi.py",
        name="index",
        engine_type="query",
        input_example="熱力學的學習目標",
        code_paths=["week_5/load_index.py", "week_5/load_model.py"],  # Include dependency
        registered_model_name="llama_index_basic" # 登記模型名稱
    )

透過模型版本追蹤數據

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

mv = client.get_model_version(
    name="llama_index_basic",
    version="3")

In [ ]:
mv.run_id

### 透過實驗來取得run_id
1. 取得實驗id

In [ ]:
client = MlflowClient()

experiment = client.get_experiment_by_name("LlamaIndex")  # or your experiment name
experiment_id = experiment.experiment_id

print(experiment_id)

2. Search runs that logged the artifact

In [ ]:
runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="attributes.status = 'FINISHED'",
    order_by=["attributes.start_time DESC"],
    max_results=10
)

選擇你要的run

In [ ]:
runs[0].info.run_id

### Find runs by dataset lineage

Because we logged the dataset input, you can search by dataset name:

In [ ]:
runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="dataset.name = 'benchmark'"
)

In [ ]:
runs

### Find runs by tags

In [ ]:
runs = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="tags.index_type = 'faiss'"
)

In [ ]:
runs

# Property Graph Index

本章節說明 **Property Graph Index** 在檢索系統中的定位，以及它與傳統檢索技術、知識圖譜之間的關係。

上週我們學會如何使用 **RAGAS Knowledge Graph** 來產生合成資料（synthetic data）。  
同樣的概念，也可以延伸到 **Retriever / Query Engine**，透過 **Graph Index** 來提升檢索品質。

---

## 一、Property Graph Index 與傳統檢索技術的關係

Property Graph Index 並不是取代 BM25 或 Semantic Search，  
而是 **吸收兩者的優點，並補足其結構與邏輯上的不足**。

---

## 1️⃣ 它像 BM25 嗎？——掌握「詞彙 / 實體」

### 相似點
- **BM25** 依賴關鍵字的「出現頻率」
- **Property Graph** 會先從文本中抽取 **實體（Entity）**
- 本質上，是一種「強化版的關鍵字抽取」

**範例比較：**

- BM25  
  - 只知道「原子」這個詞出現了 5 次
- Property Graph  
  - 知道「原子」是一個 **實體**
  - 並且在圖中是一個 **唯一節點（Node）**

### 超越點
- 解決 **關鍵字歧義問題**
  - BM25 可能混淆：
    - 「蘋果（水果）」
    - 「蘋果（公司）」
  - 在 Property Graph 中：
    - 它們是 **兩個不同節點**
    - 擁有不同屬性與關係

---

## 2️⃣ 它像 Semantic Search 嗎？——掌握「關聯與順序」

### 相似點
- **語義檢索（Vector / Embedding）**
  - 透過向量空間距離表達語意關係
- **Property Graph**
  - 透過 **邊（Edge / Relationship）** 明確描述關係

**範例比較：**

- Semantic Search  
  - 「牛頓」與「萬有引力」在向量空間中距離很近
- Property Graph  
  - 明確定義：
    ```
    (牛頓) --[提出]--> (萬有引力)
    ```

### 超越點
- **更精準掌握邏輯與順序**
- 向量檢索容易混淆主被動關係，例如：
  - 「張三打了李四」
  - 「李四打了張三」
- Property Graph 使用 **有向邊（Directed Edge）**
  - 明確區分主語與賓語
  - 可追蹤因果與時間順序

---

## 3️⃣ 三種檢索技術的整體對比

| 特性 | BM25（傳統檢索） | Vector（語義檢索） | Property Graph（圖索引） |
|---|---|---|---|
| 核心單元 | 詞頻（TF-IDF） | 數值向量（Embedding） | 節點與關係（Triplet） |
| 關聯表達 | 無（僅統計出現次數） | 隱晦（距離相近） | 顯性（明確關係） |
| 邏輯與順序 | 無 | 模糊 | 極強（有向路徑） |
| 典型缺點 | 無法理解語意 | 容易產生噪音或幻覺 | 建模與抽取成本較高 |

---

# Knowledge Graph vs Property Graph

**結論先行：**

> **Knowledge Graph 是「概念與目標」**  
> **Property Graph 是「數據模型與工程實作方式」**

---

## 二、Knowledge Graph（知識圖譜, KG）

Knowledge Graph 是一個 **廣義概念**，重點在於 **知識本身的語義連結**。

### 🎯 核心目標
- 將資訊轉換為 **機器可理解的知識**
- 強調：
  - 實體（Entity）
  - 以及實體之間的語義與邏輯關係

### 📐 標準形式
- 傳統 KG 通常遵循 **W3C 標準**
  - RDF
  - OWL
- 基本結構為 **三元組（Triple）**：

Subject - Predicate - Object


### ⚠️ 主要缺點
- 結構過於精簡
- 若想替「關係」加入額外資訊（例如時間、可信度）：
  - 必須使用 **Reification（關係實體化）**
  - 需要建立多組三元組
  - 查詢與維護成本大幅上升

---

## 三、Property Graph（屬性圖, PG）

Property Graph 是目前 **業界主流的實作模型**  
（例如：Neo4j、LlamaIndex）

### 🚀 核心優勢
- 同時支援：
  - 節點（Node）
  - 邊（Edge）
  - 屬性（Properties）

### 🧱 結構說明

#### Nodes（節點）
- 可包含：
  - Label（標籤）
  - 多組 Key-Value 屬性

#### Edges（邊）
- 具有：
  - 方向性
  - 關係名稱
  - 邊本身也能攜帶屬性

### ✅ 實際優點
- 可直接在「關係」上加資訊，例如：

```json
{
  "時間": "高一",
  "重要性": "高"
}
```
| 特性            | 傳統 Knowledge Graph（RDF） | Property Graph（LlamaIndex 採用） |
| ------------- | ----------------------- | ----------------------------- |
| 基本單位          | 三元組（Triples）            | 節點、邊、屬性                       |
| 屬性存儲          | 需額外三元組                  | 直接存在節點或邊                      |
| 查詢語言          | SPARQL                  | Cypher、Gremlin                |
| 結構複雜度         | 高，偏學術與本體推理              | 低，偏工程與 RAG                    |
| LlamaIndex 應用 | 早期使用                    | 現行主流（PropertyGraphIndex）      |


In [ ]:
import os

os.chdir("../")

from src.ollama_connection import llama_index_ollama

llm = llama_index_ollama(model='gpt-oss:120b-cloud', temperature=0)

In [ ]:
from llama_index.core import Document 
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()

docs = loader.load("week_1/08物理.pdf")

doc_text = "\n\n".join([d.get_content() for d in docs])
docs = [Document(text=doc_text)]

## (default) SimpleLLMPathExtractor
Extract short statements using an LLM to prompt and parse single-hop paths in the format (entity1, relation, entity2)

In [ ]:
from llama_index.core.indices.property_graph import SimpleLLMPathExtractor

extractor1 = SimpleLLMPathExtractor(
    llm=llm,
    max_paths_per_chunk=10,
    num_workers=4,
)

In [ ]:
SimpleLLMPathExtractor?

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [ ]:
import nest_asyncio

from llama_index.core import PropertyGraphIndex

nest_asyncio.apply()

index = PropertyGraphIndex.from_documents(
    docs,
    kg_extractors=[extractor1],
    embed_model=embed_model
)

# insert additional documents / nodes
# index.insert(docs)
# index.insert(nodes)

Get the parsed documents

In [ ]:
index.docstore.docs

In [ ]:
graph = index.property_graph_store.graph

## DynamicLLMPathExtractor
會根據可選的允許實體類型（Entity Types）和關係類型（Relation Types）列表來提取路徑（包含實體類型！）。如果未提供清單，LLM 將根據其判斷自行分配類型；如果已提供清單，它將引導 LLM 進行提取，但不會強制要求完全符合這些類型。

DynamicLLMPathExtractor 是一個用於從文本中提取結構化資訊以構建知識圖譜（Knowledge Graph）的組件。它利用 LLM 來識別實體及其關係，並具備推斷實體類型以及擴展初始本體（Ontology）的能力。

此提取器對 SimpleLLMPathExtractor 進行了以下改進：
- 檢測實體類型：不再將實體通用地標記為「entity」和「chunk」，而是檢測其具體類型。
- 接收初始本體作為輸入：可指定所需的節點（Nodes）和關係。
- 鼓勵本體擴展：透過提示詞（Prompt）設計，鼓勵系統擴展初始定義的本體。

此提取器與 SchemaLLMPathExtractor 的不同之處在於：

- 它將傳入的可選實體和關係視為「初始本體」。
- 它在提示詞中鼓勵對初始本體進行擴展。
- 它的目標是在提供引導的同時，保持知識圖譜構建的靈活性。

In [ ]:
import nest_asyncio
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor

nest_asyncio.apply()

kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=4,
    allowed_entity_types=["力學", "電磁學", "目標"],
    allowed_relation_types=["發明", "發現"],
    
)

In [ ]:
print(kg_extractor.extract_prompt.template)

In [ ]:
from llama_index.core import PropertyGraphIndex

dynamic_index = PropertyGraphIndex.from_documents(
    docs,
    kg_extractors=[kg_extractor],
    embed_model=embed_model
)

In [ ]:
dynamic_graph = dynamic_index.property_graph_store.graph

In [ ]:
dynamic_graph.get_triplets()

## SchemaLLMPathExtractor
根據允許的實體（Entities）、關係（Relationships）以及「哪些實體可以連接到哪些關係」的嚴格架構（Strict Schema）來提取路徑。

透過使用 Pydantic、LLM 的結構化輸出（Structured Outputs）以及一些巧妙的驗證機制，我們可以動態地指定架構，並針對每一條提取出的路徑進行驗證。

** 切換成OpenAI API。Ollama抽取的結果是空的 @@

### 英文文件

In [ ]:
from glob import glob
from pathlib import Path

from llama_index.core import Document 
from llama_index.readers.file import FlatReader

docs = []

for filename in glob("week_4/ragas-airline-dataset/*.md"):
    path = Path(filename)
    doc_text = FlatReader().load_data(path)
    docs.extend(doc_text)

In [ ]:
from typing import Literal

from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.llms.openai import OpenAI

from initialization import credential_init

credential_init

openai_llm = OpenAI(
    model="gpt-4o-mini",
)

entities = Literal["PERSON", "PLACE", "THING"]
relations = Literal["PART_OF", "HAS", "IS_A"]
schema = {
    "PERSON": ["PART_OF", "HAS", "IS_A"],
    "PLACE": ["PART_OF", "HAS"],
    "THING": ["IS_A"],
}

kg_extractor = SchemaLLMPathExtractor(
    llm=openai_llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=schema,
    strict=True,  # if false, will allow triplets outside of the schema
    num_workers=4,
    max_triplets_per_chunk=10,
)

In [ ]:
kg_extractor.extract_prompt.template

In [ ]:
import nest_asyncio

from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

nest_asyncio.apply()

index = PropertyGraphIndex.from_documents(
    docs,
    kg_extractors=[kg_extractor],
    embed_model=embed_model
)

In [ ]:
graph = index.property_graph_store.graph

In [ ]:
# graph.get_triplets()

## 實作單元：使用「人、事、時、地、物」構建物理課綱知識圖譜
在知識圖譜的設計中，選擇合適的**實體（Entity）**分類是關鍵。本單元採用最符合人類直覺的五種分類方式，對《高中基礎物理課綱》進行建模。

### 1. 定義 Schema（結構綱要）
我們將實體限制為「人、事、時、地、物」，並定義它們之間合法的連結關係，以確保生成的圖譜邏輯嚴謹。

In [ ]:
import os
import sys

# jupyter notebook
os.environ["PYTHONUTF8"] = "1"

In [ ]:
os.chdir("../")

In [ ]:
from typing import Literal

from llama_index.llms.openai import OpenAI
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

from initialization import credential_init

credential_init()

openai_llm = OpenAI(
    model="gpt-4o-mini",
)

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

In [ ]:
# 1. 定義實體與關係的範疇
PhysicalEntities = Literal["人", "事", "時", "地", "物"]

# 2. 建立驗證規則：定義哪些實體之間可以有哪些關係
PhysicalRelations = Literal["參與", "發生於", "包含", "操作", "規範"]

# 3. 定義驗證架構
# 例如：「人」會參與「事」；「事」會有對應的「物」或「時」
kg_validation_schema = {
    "人": ["參與", "操作"], # 例如：學生 操作 實驗器材
    "事": ["發生於", "包含", "規範"], # 例如：基礎物理一 包含 萬有引力
    "物": ["發生於"] # 例如：示範實驗器材 發生於 物理實驗室
}

# 4. 初始化提取器
kg_extractor = SchemaLLMPathExtractor(
    llm=openai_llm,
    possible_entities=PhysicalEntities,
    possible_relations=PhysicalRelations,
    kg_validation_schema=kg_validation_schema,
    strict=True # 啟動嚴格模式，確保 LLM 僅使用我們定義的標籤
)

### 2. 應用於物理課綱文件
當我們將此提取器應用於 08物理.pdf 時，系統會自動識別如下內容：

人：克卜勒、伽利略、學生、教師。

事：物質的基本交互作用、行星運動定律、補救教學。

時：每週二節、高一、高二、97年1月。

地：實驗室、國民中小學。

物：游標尺、摩擦力觀察裝置、教材

In [ ]:
from llama_index.core import Document 
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()

docs = loader.load("week_1/08物理.pdf")

doc_text = "\n\n".join([d.get_content() for d in docs])
docs = [Document(text=doc_text)]

### 3. 建立Property Graph Index

In [ ]:
import nest_asyncio

nest_asyncio.apply()

index_physics = PropertyGraphIndex.from_documents(
    docs,
    kg_extractors=[kg_extractor],
    embed_model=embed_model
)

In [ ]:
index_physics.property_graph_store.save_networkx_graph(name="week_5/kg.html")

In [ ]:
graph_physics = index_physics.property_graph_store.graph

In [ ]:
# graph_physics.nodes["9d71ab0b-e0be-4c09-baf5-3674726bc52d"]

In [ ]:
# graph_physics.get_triplets()

## Retrieval and Querying

帶有標籤的屬性圖（Labeled Property Graph）可以透過多種方式進行查詢，以取得節點與路徑。而在 **LlamaIndex** 中，我們可以**同時組合多種節點檢索方法**來使用。

---
```
# create a retriever
retriever = index.as_retriever(sub_retrievers=[retriever1, retriever2, ...])

# create a query engine
query_engine = index.as_query_engine(
    sub_retrievers=[retriever1, retriever2, ...]
)
```

如果沒有提供任何子檢索器（sub-retrievers），系統預設會使用：

- LLMSynonymRetriever
- VectorContextRetriever（前提是已啟用 embeddings）

目前支援的 Retriever 類型

LLMSynonymRetriever
根據 LLM 產生的關鍵字／同義詞進行檢索。

- VectorContextRetriever
    根據屬性圖中節點的向量嵌入（Embedding）進行檢索。

- TextToCypherRetriever
    請 LLM 根據屬性圖的 schema 產生 Cypher 查詢語句。

- CypherTemplateRetriever
    使用 Cypher 查詢模板，並由 LLM 推斷所需的查詢參數。

- CustomPGRetriever
    透過繼承方式輕鬆實作自訂的檢索邏輯。

一般情況下，你會定義一個或多個子檢索器（sub-retrievers），並將它們傳入 PGRetriever 中：

```
from llama_index.core.indices.property_graph import (
    PGRetriever,
    VectorContextRetriever,
    LLMSynonymRetriever,
)

sub_retrievers = [
    VectorContextRetriever(index.property_graph_store, ...),
    LLMSynonymRetriever(index.property_graph_store, ...),
]

retriever = PGRetriever(sub_retrievers=sub_retrievers)

nodes = retriever.retrieve("<query>")
```

## Querying

### 匹配的核心：實體名稱 (name)

當您輸入一個查詢（Query），例如「學生需要做練習嗎？」，Retriever 的運作邏輯如下：

1. 關鍵字提取：LLM 會先從查詢中提取關鍵字，例如 學生、練習。

2. 同義詞擴展：LLM 會根據上下文產生同義詞。例如針對 練習，它可能會想到 習題 或 作業。

3. 精準匹配：系統會在圖譜中尋找 name='學生' 或 name='習題' 的實體節點。

### 標籤 (label) 的角色
label（例如：'人'、'事'）在同義詞檢索中通常扮演的是**「過濾器」或「結構約束」**的角色，而不是主要搜索內容。

label 幫助 LLM 理解該實體的範疇。

在執行像 SchemaLLMPathExtractor 這種有架構限制的提取時，label 確保了關係的合法性（例如：只有 '人' 標籤的節點可以執行 '參與' 這個關係）。

### 三元組（Triplets）在檢索中的作用
當 LLM Synonym Retriever 匹配到某個 name 後，它會順藤摸瓜抓取相關的三元組。

以您的資料為例：

您搜尋：「教科書內容」。

Retriever 匹配到 name='教科書各單元教材'。

它會根據三元組關係，自動帶出：(教科書各單元教材) --[包含]--> (習題)。

最終提供給 LLM 作為參考答案的資訊，就是這整條邏輯路徑。

In [ ]:
# 之後再解釋這是幹嘛用的

synonym_retriever = LLMSynonymRetriever(
    index_physics.property_graph_store,
    llm=ollama_llm,
    # include source chunk text with retrieved paths
    include_text=True,
    # synonym_prompt=prompt,
    # output_parsing_fn=parse_fn,
    max_keywords=10,
    # the depth of relations to follow after node retrieval
    path_depth=1,
)

include_text 是一個非常關鍵的參數，它決定了檢索結果的**「厚度」**。

簡單來說，這個參數決定了檢索器在找到圖譜中的「三元組（Triplets）」後，是否要**「回頭去抓取原始的文本段落」**。

以下是詳細的差異對比：

---

1. 當 include_text=False (預設或純圖檢索)

檢索器只會回傳從圖存儲（Property Graph Store）中提取出來的結構化資訊。

- 回傳內容：僅包含實體名稱、關係以及屬性（即你之前看到的 node-relationship-node 組合）。

- 給 LLM 的參考資料：

    「學生 參與 形成性評量」、「教科書各單元教材 包含 習題」。

- **優點**：資訊非常精簡、精確，完全排除無關雜訊。

- **缺點**：如果三元組本身寫得太簡略（例如：人 --[操作]--> 物），LLM 可能會因為缺乏細節而無法回答深入的問題。

2. 當 include_text=True (圖與文本的混合檢索)

這是 LlamaIndex 強大的地方。檢索器除了抓取結構化的三元組外，還會利用節點上的 source_id 或 triplet_source_id 屬性，去 docstore 中找出產生這些三元組的原始文本塊（Source Chunks）。

回傳內容：

- 結構化三元組：如上所述的關係鏈。

- 原始文本段落：產生該三元組的那一段完整的 PDF 文字。

- 給 LLM 的參考資料：

    **圖路徑**: 「學生 參與 形成性評量」

    **原始文本**: 「...教師在評量時應顧及個別差異，對於學習較慢之學生應實施補救教學，並透過形成性評量掌握學習進度...」

- **優點**：提供豐富的上下文語境，LLM 能夠結合圖譜的「邏輯結構」與原文的「細節描述」來回答。

- **缺點**：會消耗更多的 Token，且如果原始文本很長，可能會引入一些干擾資訊。

In [ ]:
node

In [ ]:
nest_asyncio.apply()

retriever_physics = index_physics.as_retriever(
    include_text=False,  # include source text, default True
    sub_retrievers=[synonym_retriever]
)

nodes = retriever_physics.retrieve("根據課綱，學生需要參與那些項目?")

for node in nodes:
    print(node.text)

In [ ]:
# node

In [ ]:
nest_asyncio.apply()

retriever_physics = index_physics.as_retriever(
    include_text=True,  # include source text, default True
    sub_retrievers=[synonym_retriever]
)

nodes = retriever_physics.retrieve("根據課綱，學生需要參與那些項目?")

for node in nodes:
    print(node.text)

In [ ]:
query_engine_physics = index_physics.as_query_engine(
    include_text=True,
    llm=ollama_llm,
    sub_retrievers=[synonym_retriever]
)

response = query_engine_physics.query("根據課綱，學生需要參與那些項目?")

print(str(response))

## Storage

By default, storage happens using our simple in-memory abstractions - SimpleVectorStore for embeddings and SimplePropertyGraphStore for the property graph.

We can save and load these to/from disk.

中文在jupyter notebook中會出問題:

\uf9b4 是「領」字（領域的領）的一種特殊兼容碼。

    **來源**：這份 PDF 在製作時可能使用了特定的字型，導致「領」字被編碼為這個特殊碼點。
    **衝突**：在 CP950 (Big5) 的對照表中，並沒有 \uf9b4 這個碼位。即使它長得跟普通的「領」一模一樣，Windows 只要看到這個碼點，就會因為「非法多字節序列」而拒絕寫入。

嘗試使用propertygraph.py:

在命令提示字元中執行
- $env:PYTHONUTF8=1
- python -m week_5.propertygraph_save



In [1]:
from llama_index.core import StorageContext, load_index_from_storage


# 3. 儲存時，如果是在 Windows，建議顯式檢查目錄
persist_path = "storage_physics"

index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir=persist_path)
)

UnicodeDecodeError: 'cp950' codec can't decode byte 0x85 in position 22961: illegal multibyte sequence

## (default) LLMSynonymRetriever
The LLMSynonymRetriever takes the query, and tries to generate keywords and synonyms to retrieve nodes (and therefore the paths connected to those nodes).

In [ ]:
from src.ollama_connection import llama_index_ollama

model = "gpt-oss:120b-cloud"

ollama_llm = llama_index_ollama(model=model, temperature=0)

In [ ]:
from llama_index.core.indices.property_graph import LLMSynonymRetriever

# prompt = (
#     "Given some initial query, generate synonyms or related keywords up to {max_keywords} in total, "
#     "considering possible cases of capitalization, pluralization, common expressions, etc.\n"
#     "Provide all synonyms/keywords separated by '^' symbols: 'keyword1^keyword2^...'\n"
#     "Note, result should be in one-line, separated by '^' symbols."
#     "----\n"
#     "QUERY: {query_str}\n"
#     "----\n"
#     "KEYWORDS: "
# )


# def parse_fn(self, output: str) -> list[str]:
#     matches = output.strip().split("^")

#     # capitalize to normalize with ingestion
#     return [x.strip().capitalize() for x in matches if x.strip()]


synonym_retriever = LLMSynonymRetriever(
    index_physics.property_graph_store,
    llm=ollama_llm,
    # include source chunk text with retrieved paths
    include_text=False,
    # synonym_prompt=prompt,
    # output_parsing_fn=parse_fn,
    max_keywords=10,
    # the depth of relations to follow after node retrieval
    path_depth=1,
)

retriever_physics = index_physics.as_retriever(sub_retrievers=[synonym_retriever])